In [0]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout, BatchNormalization, GlobalAveragePooling2D
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint


Using TensorFlow backend.


In [0]:
num_classes=62  #although only 34 classes are used 62 have been put so as to make minimal changes in the class mapping 
img_size=28
def img_label(data_path, num_classes=62):
    data = pd.read_csv(data_path, header=None)
    data_rows = len(data)
    
    imgs = np.transpose(data.values[:,1:].reshape(data_rows, img_size, img_size, 1),axes=[0,2,1,3])  #values in csv were such that the image needs to be transposed before training
    
    labels = to_categorical(data.values[:,0], num_classes)
    
    return imgs/255, labels

In [0]:
model=Sequential()
model.add(Conv2D(32, kernel_size = 3, activation='relu', input_shape = (28, 28, 1)))
model.add(BatchNormalization())
model.add(Conv2D(32, kernel_size = 3, activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(32, kernel_size = 5, strides=2, padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(64, kernel_size = 3, activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size = 3, activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size = 5, strides=2, padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(128, kernel_size = 4, activation='relu'))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(62, activation='softmax'))

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
batch_normalization (BatchNo (None, 26, 26, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 24, 24, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 32)        25632     
_________________________________________________________________
batch_normalization_2 (Batch (None, 12, 12, 32)        128       
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 32)        0

In [0]:
data_generator_aug=ImageDataGenerator(rotation_range=30,validation_split=0.1)

In [0]:
data_generator=ImageDataGenerator(validation_split=0.1)

In [0]:
X,Y=img_label('/content/drive/My Drive/final_training_dataset.csv',62)

In [0]:
checkpoint = ModelCheckpoint('/content/drive/My Drive/modified_weights-{epoch:03d}.h5',
                                 monitor='val_acc',
                                 verbose=0,
                                 save_best_only=False,
                                 mode='auto')

In [0]:
training_data_generator = data_generator_aug.flow(X, Y,batch_size=2000, subset='training')
validation_data_generator = data_generator.flow(X, Y,batch_size=2000,subset='validation')
history = model.fit_generator(training_data_generator, steps_per_epoch=500, epochs=10, validation_data=validation_data_generator,validation_steps=1,callbacks=[checkpoint])

In [0]:
model.save('/content/drive/My Drive/new_model_yash_4.h5')

In [0]:
import cv2
import matplotlib.pyplot as plt
test_image=cv2.imread('/content/drive/My Drive/test_1/character1.jpeg',cv2.IMREAD_GRAYSCALE)
print(test_image.shape)

In [0]:
test_image=cv2.resize(test_image,(28,28))
test_image=cv2.bitwise_not(test_image)

In [0]:
plt.imshow(test_image,cmap='Greys_r')
plt.show()

In [0]:
class_mapping='0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz'
x=np.reshape(test_image,(28,28,1))/255
y=[x]
y=np.array(y)
result=np.argmax(model.predict(y))
print(result)
print(class_mapping[result])